In [4]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
# Random stuff
import random

In [5]:
VALID_MOVES = [
    (-1,-1), (-1,0), (-1,1), 
    (0,-1), (0,1), (0,0),
    (1,-1), (1,1), (1,0),
]

DIRTY_FLAG = 1
CLEAN_FLAG = 0
VACUUM_FLAG = 100

In [14]:
class ClenearModel(ap.Model):

    def setup(self):

        # Create agents (dirty floors and vacuums)
        n_dirty_floors = int(self.p['Dirt density'] * (self.p.size['X'] * self.p.size['Y']))
        self.dirty_tiles = ap.AgentList(self, n_dirty_floors)
        self.dirty_tiles.condition = DIRTY_FLAG
        
        # n_dirty_floors = int(self.p['vacums'] * (self.p.size['X'] * self.p.size['Y']))
        self.vacuums = ap.AgentList(self, int(self.p['vacums']))
        self.vacuums.condition = VACUUM_FLAG
        
        # Create grid (forest)
        self.room = ap.Grid(self, [self.p['size']['Y'], self.p['size']['X']], track_empty=True)
        self.room.add_agents(self.vacuums, [
            (int(self.p['size']['Y'])//2,int(self.p['size']['X'])//2) for i in 
            range(int(self.p['vacums']))])
        self.room.add_agents(self.dirty_tiles, random=True, empty=True)

    def step(self):
        # Start cleaning
        for vacuum in self.vacuums:
            for neighbor in self.room.neighbors(vacuum, distance=0):
                if neighbor.condition == DIRTY_FLAG:
                    neighbor.condition = CLEAN_FLAG # Neighbor cleaned
                    break
            else: # No break found during the loop
                self.room.move_by(vacuum, random.choice(VALID_MOVES))
        # Select dirty Tiles
        dirty_tiles = self.dirty_tiles.select(self.dirty_tiles.condition == 1)
        # Stop simulation if no dirty tiles is left
        if len(dirty_tiles) == 0:
            self.stop()

    def end(self):
        self.perc_ = 10/100
        self.report('Percentage of Dirty Tiles', 10/100)
        # Document a measure at the end of the simulation
#         burned_trees = len(self.agents.select(self.agents.condition == 2))
#         self.report('Percentage of burned trees',
#                     burned_trees / len(self.agents))

In [10]:
# Define parameters

parameters = {
    'Dirt density': 0.2, # Percentage of grid covered by trees
    'size': {'X': 50, 'Y': 50}, # Height and length of the grid
    'steps': 50,
    'vacums': 500
}

In [15]:
# Create single-run animation with custom colors
def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('condition')
    color_dict = {CLEAN_FLAG:'#FFFFFF', DIRTY_FLAG:'#000000', VACUUM_FLAG:'#0000FF', None:'#FFFFFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a cleaning room\n"
                 f"Time-step: {model.t}, Dirty Tiles left: "
                 f"{len(model.dirty_tiles.select(model.dirty_tiles.condition == DIRTY_FLAG))}")

fig, ax = plt.subplots()
model = ClenearModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=1))

In [16]:
model.perc_

AttributeError: ClenearModel has no attribute 'perc_'.